# Introduction to `fastplotlib`

This notebook goes through the basic components of the `fastplotlib` API, image, image updates, line plots, and scatter plots. 

**The example images are from `imageio` so you will need to install it for this example notebook. But `imageio` is not required to use `fasptlotlib`**

In [ ]:
!pip install imageio

In [ ]:
import imageio.v3 as iio

In [ ]:
from fastplotlib import Plot
from ipywidgets import VBox, HBox, IntSlider
import numpy as np

In [ ]:
# this is only for testing, you do not need this to use fastplotlib
from nb_test_utils import plot_test, notebook_finished

## Simple image

We are going to be using `jupyterlab-sidecar` to render some of the plots on the side. This makes it very easy to interact with your plots without having to constantly scroll up and down :D

In [ ]:
# create a `Plot` instance
plot = Plot()

# get a grayscale image
data = iio.imread("imageio:camera.png")

# plot the image data
image_graphic = plot.add_image(data=data, name="sample-image")

# show the plot
plot.show()

**Use the handle on the bottom right corner of the _canvas_ to resize it. You can also pan and zoom using your mouse!**

By default the origin is on the bottom left, you can click the flip button to flip the y-axis, or use `plot.camera.local.scale_y *= -1`

In [ ]:
plot.camera.local.scale_y *= -1

Changing graphic **"features"**

In [ ]:
image_graphic.cmap = "viridis"

### Slicing data

**Most features, such as `data` support slicing!**

Out image data is of shape [n_rows, n_cols]

In [ ]:
image_graphic.data().shape

In [ ]:
image_graphic.data[::15, :] = 1
image_graphic.data[:, ::15] = 1

**Fancy indexing**

In [ ]:
image_graphic.data[data > 175] = 255

Adjust vmin vmax

In [ ]:
image_graphic.cmap.vmin = 50
image_graphic.cmap.vmax = 150

In [ ]:
# testing cell, ignore
plot_test("camera", plot)

**Set the entire data array again**

Note: The shape of the new data array must match the current data shown in the Graphic.

In [ ]:
new_data = iio.imread("imageio:astronaut.png")
new_data.shape

This is an RGB image, convert to grayscale to maintain the shape of (512, 512)

In [ ]:
gray = new_data.dot([0.3, 0.6, 0.1])
gray.shape

In [ ]:
image_graphic.data = gray



reset vmin vmax

In [ ]:
image_graphic.cmap.reset_vmin_vmax()

In [ ]:
# testing cell, ignore
plot_test("astronaut", plot)

In [ ]:
plot.canvas.get_logical_size()

### Indexing plots

**Plots are indexable and give you their graphics by name**

In [ ]:
plot

In [ ]:
plot["sample-image"]

**You can also use numerical indexing on `plot.graphics`**

In [ ]:
plot.graphics

In [ ]:
plot.graphics[0]

The `Graphic` instance is also returned when you call `plot.add_<graphic_type>`.

In [ ]:
image_graphic

In [ ]:
image_graphic == plot["sample-image"]

In [ ]:
# close the sidecar
plot.sidecar.close()

### RGB images are also supported

`cmap` arguments are ignored for rgb images, but vmin vmax still works

In [ ]:
plot_rgb = Plot()

plot_rgb.add_image(new_data, name="rgb-image")

# show the plot
plot_rgb.show()

In [ ]:
plot_rgb.camera.local.scale_y *= -1

vmin and vmax are still applicable to rgb images

In [ ]:
plot_rgb["rgb-image"].cmap.vmin = 100

In [ ]:
# testing cell, ignore
plot_test("astronaut_RGB", plot_rgb)

In [ ]:
# close sidecar
plot_rgb.sidecar.close()

### Image updates

This examples show how you can define animation functions that run on every render cycle.

In [ ]:
# create another `Plot` instance
plot_v = Plot()

plot.canvas.max_buffered_frames = 1

# make some random data again
data = np.random.rand(512, 512)

# plot the data
plot_v.add_image(data=data, name="random-image")

# a function to update the image_graphic
# a plot will pass its plot instance to the animation function as an argument
def update_data(plot_instance):
    new_data = np.random.rand(512, 512)
    plot_instance["random-image"].data = new_data

#add this as an animation function
plot_v.add_animations(update_data)

# show the plot
plot_v.show()

### We can share controllers across plots

This example creates a new plot, but it synchronizes the pan-zoom controller

In [ ]:
plot_sync = Plot(controller=plot_v.controller)

data = np.random.rand(512, 512)

image_graphic_instance = plot_sync.add_image(data=data, cmap="viridis")

# you will need to define a new animation function for this graphic
def update_data_2():
    new_data = np.random.rand(512, 512)
    # alternatively, you can use the stored reference to the graphic as well instead of indexing the Plot
    image_graphic_instance.data = new_data

plot_sync.add_animations(update_data_2)

plot_sync.show(sidecar=False)

#### Keeping a reference to the Graphic instance, as shown above `image_graphic_instance`, is useful if you're creating something where you need flexibility in the naming of the graphics

### You can also use `ipywidgets.VBox` and `HBox` to stack plots. See the `gridplot` notebooks for a proper gridplot interface for more automated subplotting

In [ ]:
VBox([plot_v.show(sidecar=False), plot_sync.show(sidecar=False)])

In [ ]:
HBox([plot_v.show(sidecar=False), plot_sync.show(sidecar=False)])

In [ ]:
# close sidecar
plot_v.sidecar.close()

# Line plots

## 2D line plots

This example plots a sine wave, cosine wave, and ricker wavelet and demonstrates how **Graphic Features** can be modified by slicing!

### First generate some data.

In [ ]:
# linspace, create 100 evenly spaced x values from -10 to 10
xs = np.linspace(-10, 10, 100)
# sine wave
ys = np.sin(xs)
sine = np.dstack([xs, ys])[0]

# cosine wave
ys = np.cos(xs) + 5
cosine = np.dstack([xs, ys])[0]

# sinc function
a = 0.5
ys = np.sinc(xs) * 3 + 8
sinc = np.dstack([xs, ys])[0]

### We will plot all of it on the same plot. Each line plot will be an individual Graphic, you can have any combination of graphics on a plot.

In [ ]:
# Create a plot instance
plot_l = Plot()

# plot sine wave, use a single color
sine_graphic = plot_l.add_line(data=sine, thickness=5, colors="magenta")

# you can also use colormaps for lines!
cosine_graphic = plot_l.add_line(data=cosine, thickness=12, cmap="autumn")

# or a list of colors for each datapoint
colors = ["r"] * 25 + ["purple"] * 25 + ["y"] * 25 + ["b"] * 25
sinc_graphic = plot_l.add_line(data=sinc, thickness=5, colors = colors)

# show the plot
plot_l.show(sidecar_kwargs={"title": "lines", "layout": {'width': '800px'}})

In [ ]:
# testing cell, ignore
plot_test("lines", plot_l)

### "stretching" the camera, useful for large timeseries data

Set `maintain_aspect = False` on a camera, and then use the right mouse button and move the mouse to stretch and squeeze the view!

You can also click the **`1:1`** button to toggle this.

In [ ]:
plot_l.camera.maintain_aspect = False

### reset the plot area

In [ ]:
plot_l.auto_scale(maintain_aspect=True)

## Graphic features support slicing! :D 

In [ ]:
# indexing of colors
cosine_graphic.colors[:15] = "magenta"
cosine_graphic.colors[90:] = "red"
cosine_graphic.colors[60] = "w"

# indexing to assign colormaps to entire lines or segments
sinc_graphic.cmap[10:50] = "gray"
sine_graphic.cmap = "seismic"

# more complex indexing, set the blue value directly from an array
cosine_graphic.colors[65:90, 0] = np.linspace(0, 1, 90-65)

## You can capture changes to a graphic feature as events

In [ ]:
def callback_func(event_data):
    print(event_data)

# Will print event data when the color changes
cosine_graphic.colors.add_event_handler(callback_func)

In [ ]:
# more complex indexing of colors
# from point 15 - 30, set every 3rd point as "cyan"
cosine_graphic.colors[15:50:3] = "cyan"

In [ ]:
# testing cell, ignore
plot_test("lines-colors", plot_l)

## Graphic _data_ is itself also indexable

In [ ]:
cosine_graphic.data[10:50:5, :2] = sine[10:50:5]
cosine_graphic.data[90:, 1] = 7

In [ ]:
cosine_graphic.data[0] = np.array([[-10, 0, 0]])

In [ ]:
# testing cell, ignore
plot_test("lines-data", plot_l)

### Toggle the presence of a graphic within the scene

In [ ]:
sinc_graphic.present = False

In [ ]:
sinc_graphic.present = True

### You can create callbacks to this too, for example to re-scale the plot w.r.t. graphics that are present in the scene

In [ ]:
sinc_graphic.present.add_event_handler(plot_l.auto_scale)

In [ ]:
sinc_graphic.present = False

In [ ]:
sinc_graphic.present = True

### You can set the z-positions of graphics to have them appear under other graphics

In [ ]:
img = iio.imread("imageio:camera.png")

plot_l.add_image(img[::20, ::20], name="image", cmap="gray")

# z axis position -1 so it is below all the lines
plot_l["image"].position_z = -1
plot_l["image"].position_x = -8
plot_l["image"].position_y = -8

In [ ]:
# testing cell, ignore
plot_test("lines-underlay", plot_l)

In [ ]:
# close sidecar
plot_l.sidecar.close()

### 3D line plot

In [ ]:
# just set the camera as "3d", the rest is basically the same :D 
plot_l3d = Plot(camera='3d')

# create a spiral
phi = np.linspace(0, 30, 200)

xs = phi * np.cos(phi)
ys = phi * np.sin(phi)
zs = phi

# use 3D data
# note: you usually mix 3D and 2D graphics on the same plot
spiral = np.dstack([xs, ys, zs])[0]

plot_l3d.add_line(data=spiral, thickness=2, cmap='winter')

plot_l3d.show()

In [ ]:
plot_l3d.auto_scale(maintain_aspect=True)

In [ ]:
# testing cell, ignore
plot_test("lines-3d", plot_l3d)

In [ ]:
# close sidecar
plot_l3d.sidecar.close()

# Scatter plots

### Plot tens of thousands or millions of points

#### There might be a small delay for a few seconds before the plot shows, this is due to shaders being compiled and a few other things. The plot should be very fast and responsive once it is displayed and future modifications should also be fast!

In [ ]:
from fastplotlib import Plot
from ipywidgets import VBox, HBox, IntSlider
import numpy as np

In [ ]:
# create a random distribution of 10,000 xyz coordinates
n_points = 10_000

# if you have a good GPU go for 1.5 million points :D 
# this is multiplied by 3
#n_points = 500_000

# dimensions always have to be [n_points, xyz]
dims = (n_points, 3)

clouds_offset = 15

# create some random clouds
normal = np.random.normal(size=dims, scale=5)
# stack the data into a single array
cloud = np.vstack(
    [
        normal - clouds_offset,
        normal,
        normal + clouds_offset,
    ]
)

# color each of them separately
colors = ["yellow"] * n_points + ["cyan"] * n_points + ["magenta"] * n_points

# create plot
plot_s = Plot()

# use an alpha value since this will be a lot of points
scatter_graphic = plot_s.add_scatter(data=cloud, sizes=3, colors=colors, alpha=0.7)

plot_s.show()

### scatter graphic features work similarly to line graphic

In [ ]:
# half of the first cloud's points to red
scatter_graphic.colors[:n_points:2] = "r"

In [ ]:
# set the green value directly
scatter_graphic.colors[n_points:n_points * 2, 1] = 0.3

In [ ]:
# set color values directly using an array
scatter_graphic.colors[n_points * 2:] = np.repeat([[1, 1, 0, 0.5]], n_points, axis=0)

In [ ]:
# change the data, change y-values
scatter_graphic.data[n_points:n_points * 2, 1] += 15

In [ ]:
# set x values directly but using an array
scatter_graphic.data[n_points:n_points * 2, 0] = np.linspace(-40, 0, n_points)

In [ ]:
# close sidecar
plot_s.sidecar.close()

### You can combine VBox and HBox to create more complex layouts

This just plots everything above in a single nb output

In [ ]:
row1 = HBox([plot.show(sidecar=False), plot_v.show(sidecar=False), plot_sync.show(sidecar=False)])
row2 = HBox([plot_l.show(sidecar=False), plot_l3d.show(sidecar=False), plot_s.show(sidecar=False)])

VBox([row1, row2])

In [ ]:
# for testing, ignore
notebook_finished()